In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install PyPDF2

In [ ]:
from PyPDF2 import PdfReader, PdfWriter

def split_pdf(input_pdf, pages_per_chunk=10):
    reader = PdfReader(input_pdf)
    total_pages = len(reader.pages)
    pdf_chunks = []

    for i in range(0, total_pages, pages_per_chunk):
        writer = PdfWriter()
        for j in range(i, min(i + pages_per_chunk, total_pages)):
            writer.add_page(reader.pages[j])

        chunk_filename = f"chunk_{i // pages_per_chunk + 1}.pdf"
        with open(chunk_filename, 'wb') as chunk_file:
            writer.write(chunk_file)

        pdf_chunks.append(chunk_filename)

    return pdf_chunks

# 사용 예시
pdf_file = '/content/drive/MyDrive/설민석의 조선왕조실록.pdf'
pdf_chunks = split_pdf(pdf_file, pages_per_chunk=10)
print(f"PDF 분할 완료: {pdf_chunks}")


In [ ]:
### 30개

import requests
import uuid
import time
import json

# API URL과 Secret Key
api_url = 'YOUR_API_URL'
secret_key = 'YOUR_API_KEY'

def process_pdf(file_name, output_name):
    """OCR 처리 후 텍스트 저장"""
    # 요청 데이터 생성
    request_json = {
        'images': [
            {
                'format': 'pdf',  # PDF 형식
                'name': file_name
            }
        ],
        'requestId': str(uuid.uuid4()),
        'version': 'V2',
        'timestamp': int(round(time.time() * 1000)),
        "lang": "ko"
    }

    # 파일과 헤더 설정
    payload = {'message': json.dumps(request_json).encode('UTF-8')}
    files = [
        ('file', open(file_name, 'rb'))
    ]
    headers = {
        'X-OCR-SECRET': secret_key
    }

    # API 요청
    response = requests.post(api_url, headers=headers, data=payload, files=files)

    # 결과 처리
    if response.status_code == 200:
        ocr_result = response.json()  # JSON 응답 파싱
        extracted_text = []

        # 텍스트 추출
        for image in ocr_result.get('images', []):
            for field in image.get('fields', []):
                extracted_text.append(field.get('inferText', ''))

        # 문장 형태로 병합
        full_text = ' '.join(extracted_text).replace(' .', '.').replace(' ,', ',')

        # 텍스트 파일로 저장
        with open(output_name, 'w', encoding='utf-8') as f:
            f.write(full_text)

        print(f"OCR 결과가 '{output_name}' 파일에 저장되었습니다.")
    else:
        print(f"API 요청 실패 ({file_name}): {response.status_code}, {response.text}")

# 반복 처리 (30개 chunk 파일)
for i in range(1, 31):
    input_file = f'chunk_{i}.pdf'  # PDF 파일명 (chunk_1.pdf, chunk_2.pdf, ...)
    output_file = f'/content/drive/MyDrive/ocr_result_chunk_{i}.txt'  # 결과 저장 파일명
    print(f"Processing {input_file}...")
    process_pdf(input_file, output_file)
